# Evolutionary Strategy (ES) Algorithm

# Define objective functions

In [ ]:
import numpy as np


def f1(x, y):
    return x ** 2.0 + y ** 2.0


def f2(x, y):
    return np.sin(x) ** 2.0 + np.sin(y) ** 2.0 + 0.005 * (x ** 2.0 + y ** 2.0)


def f3(x, y):
    return 50.0 * abs(x + y) + x ** 2.0


def f4(x, y):
    return x ** 2.0 + 50.0 * y ** 2.0


def Ackley(x, y):
    """
    A multidimensional function. minimum at [0,0,..0] equals to 0. usually evaluated
    on hypercube: [-32.768, 32.768]
           x :  Each column of x represents one point
           d :  dimension
    """
    X = np.asarray([x, y])
    a = 20.0
    b = 0.2
    c = 2 * np.pi
    s = (
        -a * np.exp(-b * np.sqrt(1.0 / 2 * np.sum(np.square(X), 0)))
        - np.exp(1.0 / 2 * np.sum(np.cos(c * X), 0))
        + a
        + np.exp(1)
    )
    return s


def Levy(x, y):
    wx = (x + 3.0) / 4.0
    wy = (y + 3.0) / 4.0

    s = np.sin(np.pi * wx) ** 2 + (wy - 1) * (wy - 1) * (
        1 + np.sin(2 * np.pi * wy) ** 2
    )
    s += (wx - 1) * (wx - 1) * (1 + 10 * np.sin(np.pi * wx) ** 2)

    return s

# Plot mesh

In [ ]:
# plot mesh points
import matplotlib.pyplot as plt
from IPython import get_ipython

# for interactive plots
get_ipython().run_line_magic("matplotlib", "widget")

x = np.arange(0, 6, 1)
y = np.arange(10, 13, 1)
X, Y = np.meshgrid(x, y)

print("Array (shape): content")
print(f"x {x.shape}: {x} \ny {y.shape}: {y}")
print(f"X {X.shape}: \n{X}\nY {Y.shape}: \n{Y}")

In [ ]:
plt.scatter(X.reshape(-1), Y.reshape(-1))
plt.show()

# Plot an objective function

In [ ]:
f = Ackley

delta = 0.5
x = np.arange(-20, 20, delta)
y = np.arange(-20, 20, delta)
X, Y = np.meshgrid(x, y)
Z = f(X, Y)

# plot
fig = plt.figure(figsize=(10, 5))
ax1 = fig.add_subplot(1, 2, 1, projection="3d")
ax2 = fig.add_subplot(1, 2, 2)
ax1.plot_surface(X, Y, Z, cmap="coolwarm")
ax2.contour(X, Y, Z, cmap="coolwarm")
plt.show()

# Initializing population of size: $\mu$
- An individual contains more than the decision variables.
- Furthermore, it contains standard deviations and rotation angles.
- The decision variables are real values.

In [ ]:
def initial_population(Mu, n, lb, ub, sigma_0, n_sigma, alpha_0, n_alpha):
    """Initializes a population with decision variables and variables defining the mutation. This allows the mutations
    to adapt.

    Args:
        Mu: number of parents
        n: number of decision variables
        lb: lower limit of search domain
        ub: upper limit of search domain
        sigma_0: initial standard deviation value
        n_sigma: number of standard deviation values
        alpha_0: initial rotation angle (between 0 and pi)
        n_alpha: number of rotation angles
 
    Returns: 
        array of tuples: (object variables, standard deviations, rotation angles)
    """
    population = []

    # loop over all parent individuals
    for i in range(Mu):
        # Initialize randomly the vector of design variables.
        x = [np.random.uniform(lb, ub) for _ in range(n)]

        # Initialize the list of standard deviations.
        sigma = [sigma_0 for _ in range(n_sigma)]

        # Initialize the list of rotation angles.
        alpha = [alpha_0 for _ in range(n_alpha)]

        # Concatenate to an individual
        individual = (x, sigma, alpha)

        population.append(individual)

    return population

In [ ]:
# initialize population
pop = initial_population(
    Mu=5, n=2, lb=0, ub=10, sigma_0=1, n_sigma=2, alpha_0=0, n_alpha=1
)
print("Population:")
print(*pop, sep="\n")

# Evaluate cost of each individual
- Cost of an individual depends only on the decision variables
- We miminize cost in ES (Remember that in GA, we maximized fitness).

In [ ]:
def evaluate(population, f):
    """Computes cost value for each individual in a population
    Args:
        population: list of individuals
        f: objective function 
    Returns:
        list of cost values for each individual    
    """

    cost = []

    for individual in population:
        x, y = individual[0]  # extract only the decision variables
        cost.append(f(x, y))

    # Return list of cost values.
    return cost

In [ ]:
# Evaluate cost
G0 = initial_population(
    Mu=5, n=2, lb=0, ub=10, sigma_0=1, n_sigma=2, alpha_0=0, n_alpha=2
)
fG0 = evaluate(G0, f=f)
for i, fi in zip(G0, fG0):
    print(f"Cost of {i} = {fi:.2f}.")

# Recombine to generate offsprings ($\lambda > \mu$) from a population

In [ ]:
def recombine(population, Lambda, print_all=False):
    """Recombines individuals in a population to generate offsprings

    Args:
        population: population of individuals
        Lambda: number of offsprings
        print_all: Flag to print details
    Returns:
        list of offsprings
    """
    Mu = len(population)  # population size

    # generate n=Lambda offsprings
    recombined_population = []
    for i in range(Lambda):
        # Randomly choose two different parents
        # Improvement: Here cost based selection could be used!
        # Think roulette wheel with fitness that is ordered opposite to cost
        if Mu > 1:
            i1, i2 = np.random.choice(Mu, 2, replace=False)
        else:
            i1, i2 = (0, 0)

        individual_1 = population[i1]
        individual_2 = population[i2]

        # Discrete recombination of object variables (decision variables)
        # Each decision variable of offspring comes from one of the two parents.
        # Improvement: Instead of discrete recombination, interpolation can be used!
        x_1 = individual_1[0]
        x_2 = individual_2[0]
        x_new = []
        assert len(x_1) == len(x_2)
        for j in range(len(x_1)):
            # For each component of the object variables' vector, choose randomly the parent.
            r = np.random.choice([0, 1])
            # Take the value from parent 1 or 2.
            if r == 0:
                x_new.append(x_1[j])
            elif r == 1:
                x_new.append(x_2[j])

        # Intermediate recombination of standard deviations
        sigma_1 = np.asarray(individual_1[1])
        sigma_2 = np.asarray(individual_2[1])
        sigma_new = (sigma_1 + sigma_2) / 2.0

        # Intermediate recombination of rotation angles
        alpha_1 = np.asarray(individual_1[2])
        alpha_2 = np.asarray(individual_2[2])
        alpha_new = (alpha_1 + alpha_2) / 2.0

        # add new decision variables, sigma and alpha
        new_individual = (x_new, sigma_new, alpha_new)
        recombined_population.append(new_individual)

        if print_all:
            p1x, p1y = individual_1[0]
            p2x, p2y = individual_2[0]
            cx, cy = x_new
            print(
                f"Combination of decision variables ({p1x:.2f}, {p1y:.2f}), ({p2x:.2f}, {p2y:.2f}) gives ({cx:.2f}, {cy:.2f})"
            )

    return recombined_population

In [ ]:
# print initial population
G0 = initial_population(
    Mu=5, n=2, lb=0, ub=10, sigma_0=1, n_sigma=2, alpha_0=0, n_alpha=2
)
C0 = recombine(G0, Lambda=7, print_all=True)

# Mutate individuals based on standard deviation and correlation between variables

In [ ]:
def mutate(population, ax, lb, ub, plotting=False):
    """Mutates offspring population. Also plots mutation profile if plotting = True

    Args:
        population: list of individuals
        ax: axis for plotting mutation profile
        lb: lower bound of search space
        ub: upper bound of search space
        plotting: flag to activate plotting mutation profile

    Returns: list of mutated individuals        
    """
    mutated_population = []

    # Number of decision variables
    dim = len(population[0][0])

    # Default parameters for Evolution Strategy.
    # Learning rate when one global standard deviation is given
    tau_0 = 1.0 / np.sqrt(dim)

    # Learning rates when multiple standard deviations are used
    tau = 1.0 / np.sqrt(2 * np.sqrt(dim))
    tau_prime = 1.0 / np.sqrt(2 * dim)
    beta = 0.0873  # 5 degree

    # Loop over all individuals in the population.
    for individual in population:
        # Get the vector of object variables, standard deviations and rotation angles.
        x, sigma, alpha = individual
        n_sigma = len(sigma)
        n_alpha = len(alpha)

        # Generation of a random number from standard normal distribution (mean=0, standard deviation=1.0).
        N = np.random.normal(0, 1)

        # Choose a self-adaptation strategy
        if n_sigma == 1:
            # Single step size strategy
            sigma = [sigma[0] * np.exp(tau_0 * N)]
        else:
            # Treat each sigma separately
            for i in range(n_sigma):
                N_i = np.random.normal(0, 1)
                sigma[i] = sigma[i] * np.exp(tau_prime * N + tau * N_i)

        # Mutate rotation angles. Not relevant until number of decision variables (n) >= 2.
        for i in range(n_alpha):
            alpha[i] = alpha[i] + beta * np.random.normal(0, 1)

        # Construct covariance matrix based on the mutated standard deviations and rotation angles.
        C = np.zeros(shape=(dim, dim))
        if n_sigma == 1:
            for i in range(dim):
                C[i][i] = sigma[0] ** 2.0
        elif n_sigma == dim:
            for i in range(dim):
                C[i][i] = sigma[i] ** 2.0

            # Improvement: Implement for higher dimensions
            if n_alpha == 1 and dim == 2:
                # Construct rotation matrix.
                R = np.array(
                    [
                        [np.cos(alpha[0]), -np.sin(alpha[0])],
                        [np.sin(alpha[0]), np.cos(alpha[0])],
                    ]
                )
                # Rotate covariance matrix
                C = np.dot(np.dot(R, C), np.linalg.inv(R))
            else:
                raise ("Not implemented for multiple alpha and dim != 2")

        # Plot lines of equal probablity density to place an offspring.
        if plotting:
            plot_eq_prob_density(x, C, ax, level=1, lb=lb, ub=ub)

        # Mutate object variables using multivariate normal distribution.
        mean = np.zeros(dim)
        x = np.array(x) + np.random.multivariate_normal(mean, C)
        x = x.tolist()

        mutated_population.append((x, sigma, alpha))

    return mutated_population

# Select $\mu$ best individuals based on cost from $\lambda$ parents.

In [ ]:
def select(population, cost, Mu):
    """Selects 'Mu' best individuals from 'Lambda' number of new generation.
    Selection stage in brackets: Mu -> (Lambda -> Mu)
    Args:
        population: list of individuals
        cost: cost of population
        Mu: number of best individuals selected
    Returns: list of best individuals
    """
    new_population = []
    # Sort according to the minimal value of the objective function.
    min_cost = np.argsort(cost)

    # Select Mu best offspring to create new parent population.
    # Improvement: make the selection stochastic.
    # Lower the cost, higher the probability of selection.
    for i in range(Mu):
        new_population.append(population[min_cost[i]])

    return new_population

In [ ]:
# Example selection
pop = ["a", "b", "c", "d"]
cost = [2, 3, 1, 10]
selected = select(pop, cost, Mu=2)
print(f"From population {pop} with cost {cost},\n{selected} are selected.")

# Summary of generation count ($\mu$ < $\lambda$)
- Recombination: $\mu$ parents to $\lambda$ children
- Mutation: $\lambda$ to $\lambda$ individuals
- Selection: $\lambda$ individuals to $\mu$ parents for the next iteration.

# Plotting functions needed to demonstrate the ES algorithm.

In [ ]:
# Plotting function points corresponding to the positions of parent individuals.
def plot_parents(population, ax, color="blue", alpha=1.0, face=True):
    for i in range(len(population)):
        x, y = population[i][0]
        if face:
            ax.scatter(x, y, color=color, s=70, alpha=alpha)
        else:
            ax.scatter(x, y, color=color, s=70, facecolors="none", alpha=alpha)

In [ ]:
# Plotting function points corresponding to the positions of offspring individuals.
def plot_offspring(population, ax, alpha=1.0, face=True):
    plot_parents(population, ax, color="red", alpha=alpha, face=face)

In [ ]:
# Plotting function convergence of the objective (cost) function.
def plot_convergence(t, history, ax):
    ax.plot(range(0, t + 2), history)

In [ ]:
# Function returning maximal standard deviation in population.
def get_max_sigma(population):
    sigma = [max(individual[1]) for individual in population]
    return max(sigma)

In [ ]:
# Plotting function lines of equal probability density to place an offspring (just for 2D cases).
def plot_eq_prob_density(pos, C, ax, level, lb, ub):
    C = np.linalg.inv(C)

    delta = 0.1
    x = np.arange(lb, ub, delta)
    y = np.arange(lb, ub, delta)
    X, Y = np.meshgrid(x, y)

    # ellipsoid function centered around each individual
    Z = (
        C[0][0] * (X - pos[0]) ** 2
        + C[1][0] * (X - pos[0]) * (Y - pos[1])
        + C[0][1] * (X - pos[0]) * (Y - pos[1])
        + C[1][1] * (Y - pos[1]) ** 2
    )

    ax.contour(X, Y, Z, levels=[level, level + 0.0001], alpha=0.25)

# Minimize an objective (cost) using Evolutionary Strategy

In [ ]:
import time
import copy as cp


def ES(
    Mu,
    Lambda,
    f,
    sigma_0=0.5,
    n_sigma=1,
    alpha_0=0,
    n_alpha=0,
    selection=",",
    t_max=20,
    lb=-20.0,
    ub=20.0,
    print_parents=False,
):
    """Evolutionary Strategy algorithm.

    Usage:
    # Requires fig, ax1, ax2, ax3 defined before calling this function.
    fig = plt.figure(figsize=(10, 6))
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(222)
    ax3 = fig.add_subplot(224)

    # In a different Jupyter cell, run ES
    ES(...)


    Args:
        Mu: Parent population size
        Lambda: Offspring population size
        sigma_0: Initial step size
        n_sigma: Number of standard deviations (step sizes)
        alpha_0: Initial value of the rotation angle
        n_alpha: Number of rotation angles
        selection: selection mechanism. ',': (Mu, Lambda); '+': (Mu+Lambda)
        t_max: number of iterations
        lb: lower limit
        ub: upper limit
        verbose: flag for some prints
    """
    # Resolution of the plot.
    delta = 0.1
    x = np.arange(lb, ub, delta)
    y = np.arange(lb, ub, delta)
    X, Y = np.meshgrid(x, y)
    Z = f(X, Y)

    # Dimensionality of the optimization problem.
    n = 2

    cost_history = []
    sigma_history = []

    # Initialization of the parent population.
    pop = initial_population(Mu, n, -20.0, 20.0, sigma_0, n_sigma, alpha_0, n_alpha)
    # Evaluation of the initial parent population.
    cost = evaluate(pop, f)

    # Tracking of cost values and maximal step sizes.
    cost_history.append(min(cost))
    sigma_history.append(get_max_sigma(pop))

    parents = cp.deepcopy(pop)

    # Note that fig, ax1, ax2, ax3 need to be globally defined.

    # Main optimization loop.
    for t in range(t_max):
        # Plotting of figures.
        ax1.clear()
        ax2.clear()
        ax3.clear()

        fig.suptitle("Iteration = {0:>3}".format(t + 1))
        ax1.set_title("Contour plot of the objective function")
        ax2.set_title("Convergence of the objective function")
        ax3.set_title("Maximal step size")

        ax1.set_xlim([lb, ub])
        ax1.set_ylim([lb, ub])
        ax1.grid()
        ax1.set_xlabel("x")
        ax1.set_ylabel("y", rotation=0)
        #             ax1.contour(X, Y, Z, cmap=cm.coolwarm, antialiased=False)  # faster
        ax1.contour(X, Y, Z, cmap="coolwarm")

        # Recombination.
        pop = recombine(pop, Lambda)

        # Mutation.
        pop = mutate(pop, ax1, lb, ub, plotting=True)
        # Plotting of the offspring population.
        plot_offspring(pop, ax1)

        # Selection.
        if selection == "+":  # (Mu + Lambda) strategy
            # Evaluation of the offspring and parent population.
            cost = evaluate(pop + parents, f)
            pop = select(pop + parents, cost, Mu)
        elif selection == ",":  # (Mu, Lambda) strategy
            # Evaluation of the offspring population.
            cost = evaluate(pop, f)
            pop = select(pop, cost, Mu)

        parents = cp.deepcopy(pop)
        if print_parents:
            print(parents)
        plot_parents(pop, ax1)

        # Convergence plot.
        cost_history.append(min(cost))
        plot_convergence(t, cost_history, ax2)

        # Plot evolution of maximal sigma.
        sigma_history.append(get_max_sigma(pop))
        plot_convergence(t, sigma_history, ax3)

        # output to figure
        fig.canvas.draw()
        fig.canvas.flush_events()
        time.sleep(1)

# Using , selection strategy

In [ ]:
fig = plt.figure(figsize=(10, 6))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(224)

In [ ]:
# For visualization of the process, see above
ES(
    Mu=5,
    Lambda=35,
    f=Ackley,
    selection=",",
    sigma_0=1,
    n_sigma=2,
    alpha_0=0,
    n_alpha=1,
)

# Using + selection strategy

In [ ]:
fig = plt.figure(figsize=(10, 6))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(224)

In [ ]:
# For visualization of the process, see above
ES(
    Mu=5,
    Lambda=35,
    f=Ackley,
    selection="+",
    sigma_0=1,
    n_sigma=2,
    alpha_0=0,
    n_alpha=1,
)

# Exercises:
- Is + better than , strategy for the objective functions? How do we verify?
- Minimize Levy function
- Implement cost based selection instead of random selection used here. Does it improve convergence?